In [1]:
import polars as pl
import pandas as pd
import re

from tools.josiedataprep.preparation_functions import _create_set_with_bag_semantic, _create_set_with_set_semantic

In [2]:
from tools.sloth.sloth import sloth
from tools.utils.table import from_pandas

In [3]:
# root_dir = '/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mbag'
root_dir = '/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mset'


tables_csv_dir =        '/home/nanni/unimore/tesi-magistrale/data/turl_sloth_wikitables/tables-subset/csv'
josie_sloth_ids_file =  root_dir + '/josie_sloth_ids.csv'
tables_file =           root_dir + '/tables.set'

In [4]:
josie_sloth_ids = pd.read_csv(josie_sloth_ids_file)

In [5]:
# josie_res = pd.read_csv('/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mbag/result_k_5.csv')[['query_id', 'results']]

josie_res = pd.read_csv('/home/nanni/unimore/tesi-magistrale/data/josie-tests/n45673_mset/result_k_5.csv')[['query_id', 'results']]
josie_res

,query_id,results
0,47384,s38722o1158s56253o603s18772o540s48260o56s45737o55
1,68509,s69203o45s68747o45s38674o43s11177o34s8287o28
2,24796,s69310o29s32525o24s54512o23s49194o22s25478o22
3,2637,s40234o46s78670o46s7633o45s22982o45s77978o44
4,62436,s8819o29s86983o29s45911o29s2586o29s87353o29
...,...,...
89,3362,s45755o33s77503o33s47295o33s8178o32s9564o32
90,25305,s31472o73s1893o73s61522o73s62689o73s84146o73
91,79547,s40234o27s85111o27s3416o27s61368o27s25960o27
92,83910,s56686o95s64367o95s40010o95s3069o66s62588o66


In [18]:
from pprint import pprint


print('query_id', 'set_id ', 'josie_o', 'my_o', 'sloth')
for row in josie_res.itertuples(index=False):
    qid = row[0]
    if type(row[1]) != str:
        continue
    res = row[1]
    sids, overlaps = re.findall(r'\d+', row[1])[::2], re.findall(r'\d+', row[1])[1::2]
    s_id1 = josie_sloth_ids[josie_sloth_ids['josie_id'] == qid]['sloth_id'].values[0]
    
    for sid, o in zip(sids, overlaps):                
        s_id2 = josie_sloth_ids[josie_sloth_ids['josie_id'] == int(sid)]['sloth_id'].values[0]

        df1 = pd.read_csv(tables_csv_dir + '/' + s_id1)
        df2 = pd.read_csv(tables_csv_dir + '/' + s_id2)
        
        # set1 = _create_set_with_bag_semantic(df1)
        # set2 = _create_set_with_bag_semantic(df2)
        
        set1 = _create_set_with_set_semantic(df1)
        set2 = _create_set_with_set_semantic(df2)

        tab1 = from_pandas(df1)
        tab2 = from_pandas(df2)

        mymetr = []
        res = sloth(tab1.columns, tab2.columns, verbose=False, metrics=mymetr)
        largest_ov_sloth = mymetr[-2]
        print(qid, '\t', sid, '\t', o, '\t', len(set1.intersection(set2)), '\t', largest_ov_sloth)
        if abs(len(set1.intersection(set2)) - int(o)) not in (0, 1):
            print('Errore ', s_id1, s_id2)

query_id set_id  josie_o my_o sloth
47384 	 38722 	 1158 	 1158 	 1488
47384 	 56253 	 603 	 603 	 490
47384 	 18772 	 540 	 540 	 427
47384 	 48260 	 56 	 56 	 86
47384 	 45737 	 55 	 55 	 84
68509 	 69203 	 45 	 45 	 40
68509 	 68747 	 45 	 45 	 33
68509 	 38674 	 43 	 43 	 33
68509 	 11177 	 34 	 34 	 24
68509 	 8287 	 28 	 28 	 20
24796 	 69310 	 29 	 29 	 323
'Timed Out'
24796 	 32525 	 24 	 24 	 None
'Timed Out'
24796 	 54512 	 23 	 23 	 None


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x77a1c46fa0b0>>
Traceback (most recent call last):
  File "/home/nanni/.pyenv/versions/3.10.13/envs/tesi3.10/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
  File "/home/nanni/.pyenv/versions/3.10.13/envs/tesi3.10/lib/python3.10/site-packages/timeout_decorator/timeout_decorator.py", line 69, in handler
    _raise_exception(timeout_exception, exception_message)
  File "/home/nanni/.pyenv/versions/3.10.13/envs/tesi3.10/lib/python3.10/site-packages/timeout_decorator/timeout_decorator.py", line 45, in _raise_exception
    raise exception()
timeout_decorator.timeout_decorator.TimeoutError: 'Timed Out'


In [7]:
len(res), len(res[0]), len(res[0][0]), len(res[0][0][0]), len(res[0][0][1][0]), len(res[0][0][1][2])

IndexError: string index out of range

In [ ]:
res[0][0]

([(0, 0), (2, 2), (1, 1)],
 [('Constituency', 'MP', 'Party'),
  ('Aberavon', 'John Morris', 'Labour'),
  ('Aberdeen, North', 'Robert Hughes', 'Labour'),
  ('Aldershot', 'Julian Critchley', 'Conservative'),
  ('Aldridge-Brownhills', 'Richard Shepherd', 'Conservative'),
  ('Alyn and Deeside', 'Barry Jones', 'Labour'),
  ('Amber Valley', 'Phillip Oppenheim', 'Conservative'),
  ('Antrim, East', 'Roy Beggs', 'Official Unionist'),
  ('Antrim, North', 'Rev. Ian Paisley', 'Democratic Unionist'),
  ('Antrim, South', 'Clifford Forsythe', 'Official Unionist'),
  ('Arundel', 'Michael Marshall', 'Conservative'),
  ('Ashfield', 'Frank Haynes', 'Labour'),
  ('Ashford', 'Keith Speed', 'Conservative'),
  ('Ashton-under-Lyne', 'Robert Sheldon', 'Labour'),
  ('Aylesbury', 'Timothy Raison', 'Conservative'),
  ('Ayr', 'Hon. George Younger', 'Conservative'),
  ('B', 'B', 'B'),
  ('Banbury', 'Tony Baldry', 'Conservative'),
  ('Barking', 'Jo Richardson', 'Labour'),
  ('Barnsley, East', 'Terry Patchett', 'Labo

In [8]:
df1 = pd.read_csv(tables_csv_dir + f'/439388-2')
df2 = pd.read_csv(tables_csv_dir + f'/440072-2')

In [16]:
tab1 = from_pandas(df1)
tab2 = from_pandas(df2)
mymetr = []
results, metrics = sloth(tab1.columns, tab2.columns, verbose=True, metrics=mymetr)

Detected 9 seed(s) in 0.0075070858001708984 seconds.
Generated 1 candidate(s) in 3.266334533691406e-05 seconds.
Verified 1 candidate(s) in 0.0019483566284179688 seconds.
Detected 1 largest overlap(s) in 0.002001523971557617 seconds.
Size of the detected largest overlap(s): 3 columns, 496 rows, 1488 cells.
Total elapsed time: 0.01033473014831543 seconds.
                       0                 1             2
0               Aberavon       John Morris        Labour
1        Aberdeen, North     Robert Hughes        Labour
2              Aldershot  Julian Critchley  Conservative
3    Aldridge-Brownhills  Richard Shepherd  Conservative
4       Alyn and Deeside       Barry Jones        Labour
..                   ...               ...           ...
491              Wrexham        John Marek        Labour
492              Wycombe       Ray Whitney  Conservative
493                    Y                 Y             Y
494               Yeovil     Paddy Ashdown       Liberal
495              

In [14]:
results, metrics = results[0]

In [17]:
metrics

[9,
 0.0075070858001708984,
 'e',
 None,
 1.239776611328125e-05,
 1,
 3.266334533691406e-05,
 1,
 0.0019483566284179688,
 1,
 3,
 496,
 1488,
 0.01033473014831543]

In [ ]:
df1

,Constituency,MP,Party
0,Constituency,MP,Party
1,Aberavon,John Morris,Labour
2,"Aberdeen, North",Robert Hughes,Labour
3,"Aberdeen, South",Frank Doran,Labour
4,Aldershot,Julian Critchley,Conservative
...,...,...,...
669,Wyre Forest,Anthony Coombs,Conservative
670,Y,Y,Y
671,Yeovil,Paddy Ashdown,Liberal
672,Ynys Mon,Ieuan Wyn Jones,Plaid Cymru


In [ ]:
df2

,Constituency,MP,Party
0,Constituency,MP,Party
1,Aberavon,John Morris,Labour
2,"Aberdeen, North",Robert Hughes,Labour
3,"Aberdeen, South",Gerald Malone,Conservative
4,Aldershot,Julian Critchley,Conservative
...,...,...,...
669,Wyre Forest,Esmond Bulmer,Conservative
670,Y,Y,Y
671,Yeovil,Paddy Ashdown,Liberal
672,Ynys Mon,Keith Best,Conservative
